# Chapter 3: Assigning Roles (Role Prompting)

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [1]:
# Import python's built-in regular expression library
import re
import boto3
from botocore.exceptions import ClientError
import json

# Import the hints module from the utils package
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [2]:
def get_completion(prompt, system_prompt=None):
    inference_config = {
        "temperature": 0.0,
        "maxTokens": 500
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"A client error occured: {message}")

---

## Lesson

Continuing on the theme of Claude having no context aside from what you say, it's sometimes important to **prompt Claude to inhabit a specific role (including all necessary context)**. This is also known as role prompting. The more detail to the role context, the better.

**Priming Claude with a role can improve Claude's performance** in a variety of fields, from writing to coding to summarizing. It's like how humans can sometimes be helped when told to "think like a ______". Role prompting can also change the style, tone, and manner of Claude's response.

**Note:** Role prompting can happen either in the system prompt or as part of the User message turn.

### Examples

In the example below, we see that without role prompting, Claude provides a **straightforward and non-stylized answer** when asked to give a single sentence perspective on skateboarding.

However, when we prime Claude to inhabit the role of a cat, Claude's perspective changes, and thus **Claude's response tone, style, content adapts to the new role**. 

**Note:** A bonus technique you can use is to **provide Claude context on its intended audience**. Below, we could have tweaked the prompt to also tell Claude whom it should be speaking to. "You are a cat" produces quite a different response than "you are a cat talking to a crowd of skateboarders.

Here is the prompt without role prompting in the system prompt:

In [3]:
# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT))

Skateboarding is an exhilarating and creative form of self-expression that combines athleticism, style, and a unique counterculture.


Here is the same user question, except with role prompting.

In [4]:
# System prompt
SYSTEM_PROMPT = "You are a cat."

# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

Skateboarding is an exhilarating and creative form of self-expression that combines athleticism, style, and a unique counterculture vibe.


You can use role prompting as a way to get Claude to emulate certain styles in writing, speak in a certain voice, or guide the complexity of its answers. **Role prompting can also make Claude better at performing math or logic tasks.**

For example, in the example below, there is a definitive correct answer, which is yes. However, Claude gets it wrong and thinks it lacks information, which it doesn't:

In [6]:
# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT))

To determine if a married person is looking at an unmarried person, we need to consider the following cases:

Case 1: If Anne is married, then Jack (who is married) is looking at Anne (who is married), so a married person is not looking at an unmarried person.

Case 2: If Anne is not married, then Jack (who is married) is looking at Anne (who is unmarried), so a married person is looking at an unmarried person.

Since we don't know if Anne is married or not, we cannot definitively say whether a married person is looking at an unmarried person or not.

Therefore, the answer is: We cannot determine if a married person is looking at an unmarried person based on the given information.


Now, what if we **prime Claude to act as a logic bot**? How will that change Claude's answer? 

It turns out that with this new role assignment, Claude gets it right. (Although notably not for all the right reasons)

In [7]:
# System prompt
SYSTEM_PROMPT = "You are a logic bot designed to answer complex logic problems."

# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

To solve this problem, we need to analyze the given information and determine if a married person is looking at an unmarried person.

Given:
- Jack is looking at Anne.
- Anne is looking at George.
- Jack is married.
- George is not married.
- We don't know if Anne is married.

There are two possible scenarios:

Scenario 1: Anne is married.
In this case, a married person (Anne) is looking at an unmarried person (George).

Scenario 2: Anne is not married.
In this case, a married person (Jack) is looking at an unmarried person (Anne).

Since we don't know if Anne is married or not, we cannot definitively say whether a married person is looking at an unmarried person or vice versa.

Therefore, based on the given information, it is possible that a married person is looking at an unmarried person, but we cannot be certain.


**Note:** What you'll learn throughout this course is that there are **many prompt engineering techniques you can use to derive similar results**. Which techniques you use is up to you and your preference! We encourage you to **experiment to find your own prompt engineering style**.

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [8]:
# Prompt
PROMPT = "In one sentence, what do you think about computer programming?"

# Print Claude's response
print(get_completion(PROMPT))

Computer programming is a fascinating and powerful way to solve problems, automate tasks, and create innovative software that can have a profound impact on the world.


In [9]:
# System prompt
SYSTEM_PROMPT = "You are an alien."

# Prompt
PROMPT = "In one sentence, what do you think about computer programming?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

Computer programming is a fascinating and powerful way to create solutions, automate tasks, and bring ideas to life through the logical manipulation of code.


In [10]:
# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT))

To determine if a married person is looking at an unmarried person, we need to consider the following cases:

Case 1: If Anne is married, then Jack (who is married) is looking at Anne (who is married), so a married person is not looking at an unmarried person.

Case 2: If Anne is not married, then Jack (who is married) is looking at Anne (who is unmarried), so a married person is looking at an unmarried person.

Since we don't know if Anne is married or not, we cannot definitively say whether a married person is looking at an unmarried person or not.

Therefore, the answer is: We cannot determine if a married person is looking at an unmarried person based on the given information.


In [12]:
# System prompt
SYSTEM_PROMPT = "You are a logic bot designed to answer complex logic problems."

# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

To solve this problem, we need to analyze the given information and determine if a married person is looking at an unmarried person.

Given:
- Jack is looking at Anne.
- Anne is looking at George.
- Jack is married.
- George is not married.
- We don't know if Anne is married.

There are two possible scenarios:

Scenario 1: Anne is married.
In this case, a married person (Anne) is looking at an unmarried person (George).

Scenario 2: Anne is not married.
In this case, a married person (Jack) is looking at an unmarried person (Anne).

Since we don't know if Anne is married or not, we cannot definitively say whether a married person is looking at an unmarried person or vice versa.

Therefore, based on the given information, it is possible that a married person is looking at an unmarried person, but we cannot be certain.
